In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [5]:
import numpy as np

In [6]:
import tensorflow as tf
import pickle
from emotion_classifer import EmotionClassifier

In [7]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [8]:
def extract_text(pdf_folder):
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            with open(pdf_path, 'rb') as file:
                reader = PdfReader(file)
                text = ''
                for page in reader.pages:
                    text += page.extract_text() + '\n'
                documents.append({"id": filename, "content": text.strip()})
    
    return documents

pdf_folder = 'E:\\Mental Health Chatbot\\Mental-Health-Chatbot\\docs'
documents = extract_text(pdf_folder)

#documents

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
convo_data = [doc["content"] for doc in documents]
vectorstore = FAISS.from_texts([doc["content"] for doc in documents], embeddings)

In [10]:
vectorstore

In [31]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [32]:
llm = OllamaLLM(model = "llama2")

In [37]:
template ='''
You are assisting with the Diagnosis of Thought methodology.

Start by separating the objective truth or facts from the subjective thoughts in the user-provided statement.
Do not mention this in the response

For the second step, provide reasoning that both supports the subjective thought and contradicts it.
Answer in the following way:
"You are feeling this way because (supportive reasoning), but (contradictive reasoning)."

Now, move on to the Schema Analysis stage. Identify any underlying schemas or patterns that might influence the 
approach or thoughts related to the statement.
Do not mention this in the response

Using all this information, identify the cognitive distortion.

Now, formulate an empathetic response, comforting the user, based on all the information you have gathered.
Implement a conversational flow by asking follow-up questions to better understand their situation. Offer any coping 
techniques to help them feel better about their current state. Kep the conversation open and make them feel welcome 
to talk about their feelings.

Here is the Mental Health Data: {therapy_data}

Here is the conversation history: {context}

User's Message: {input}

Tone of the User's Message : {sentiment}

User's current emotion : {emotion}

Probelms the user is quite possibly struggling with: {problem}


Answer:

'''

prompt = ChatPromptTemplate.from_template(template)

In [38]:
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
def handle_conversation():

    print("Mental Health Chatbot")
    
    with open('E:\\Mental Health Chatbot\\Mental-Health-Chatbot\\tokenizer.pkl', 'rb') as handle:
        tokenizer_sentiment = pickle.load(handle)

    with open('E:\\Mental Health Chatbot\\Mental-Health-Chatbot\\tokenizer_five.pkl', 'rb') as handle:
        tokenizer_emotion = pickle.load(handle)  

    max_length = 40
    model_path_sentiment = r'E:\Mental Health Chatbot\Mental-Health-Chatbot\my_model.h5'
    model_path_emotion = r'E:\Mental Health Chatbot\Mental-Health-Chatbot\my_model_five.h5'

    sentiment_classifier = EmotionClassifier(model_path_sentiment, tokenizer_sentiment, max_length)
    emotion_classifier = EmotionClassifier(model_path_emotion, tokenizer_emotion, max_length)

    context = " "
    print("Hey! How are you feeling today. You can type \"bye\" whenever you want to exit the chat")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "bye":
            print("It was great talking to you and I hope that I was able to help you")
            break
        predicted_class_emotion = emotion_classifier.emotion_classification(user_input)
        sentiment, prediction_dict = sentiment_classifier.sentiment_classification(user_input)
        high_confidence_emotions = [key for key, value in prediction_dict.items() if value > 0.4]
        
        result = retrieval_chain.invoke({"therapy_data": vectorstore, "context": context, "input": user_input, "sentiment": sentiment, "emotion": predicted_class_emotion, "problem": high_confidence_emotions})
        response = result['answer']
       
        print(f"MindMate: {response}")
        context += f"\nUser: {user_input}\nAI: {result['answer']}"
        #print(context)

In [41]:
handle_conversation()

Mental Health Chatbot


Hey! How are you feeling today. You can type "bye" whenever you want to exit the chat


You:  i feel really sad, i get bullied in school and dont know to deal wit it


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 876ms/step
MindMate: Dear User,

Thank you for reaching out and sharing your concerns. It takes a lot of courage to open up about personal struggles, and I appreciate your trust in me. Please know that you are not alone in this struggle, and there are many resources available to help you cope with bullying in school.

Firstly, it's important to recognize that bullying is never okay and can have serious consequences for both the bully and the victim. If you are being bullied, please know that you don't deserve to be treated this way, and you have the right to feel safe and respected in school.

Here are some strategies that may help you deal with bullying:

1. Ignore the bully: In many cases, bullies are looking for a reaction from their victims. By ignoring them or showing no emotion, you can take away their power to upset you.
2. Use positive self-talk: Talking to yourself in a positive and supportive way can help boos

You:  none of the teachers are listening to me. I dont want to live anymore


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
MindMate: Thank you for reaching out for help. It sounds like you are experiencing some difficulties with your school and teachers not taking action about the bullying issue. I'm here to listen and provide support, but I cannot make any promises or guarantees of resolution. However, I can offer some suggestions on how to approach the situation:

1. Document everything: Keep a record of all incidents of bullying, including dates, times, locations, and details of what happened. This can be helpful in case you need to provide evidence to teachers or school administrators.
2. Talk to your teachers again: It's possible that your teachers are not aware of the extent of the problem or the impact it's having on you. Try scheduling a meeting with them and providing them with your documentation. Be clear and direct about what you need from them in terms of support and action.
3. Involve school administrators: If talking 

You:  i think i will try these things out. Thank you for your help


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
MindMate: Dear User,

Thank you for reaching out and seeking help. It's great to hear that you are willing to try out some of the suggestions I provided. Remember, mental health care is a journey, and it's important to be patient and kind to yourself throughout the process. Don't hesitate to reach out if you have any questions or need further support.

Remember, anger is a normal emotion, but it can also be overwhelming at times. It's important to find healthy ways to manage your emotions, such as through exercise, meditation, or talking to a trusted friend or family member.

As for the problems you are struggling with, it's important to remember that you are not alone. Many people struggle with similar issues, and there is always hope for improvement. Don't be afraid to seek professional help if you need it, and don't hesitate to reach out to me or other resources for support.

Remember, you are strong and cap

You:  bye


It was great talking to you and I hope that I was able to help you
